## 1. Configuration

Refer to `README.md` for installation instructions.

Let's start off by sourcing secrets and defining the locations of the various required services:

In [ ]:
# source secrets
source .env

## **2. List TES instances**

Let's see what TES instances we have defined:

In [ ]:
unset TES_INSTANCES
declare -A TES_INSTANCES
while IFS=',' read -r KEY URL; do
    TES_INSTANCES["$KEY"]=$URL
done < .tes_instances

for KEY in "${!TES_INSTANCES[@]}"; do
    echo "$KEY: ${TES_INSTANCES[$KEY]}"
done

## 3. Executing workflows via the TES network

Based off a [previous demonstrator](https://github.com/elixir-cloud-aai/elixir-cloud-demos) showcasing the [cwl-tes](https://github.com/ohsu-comp-bio/cwl-tes), we will demonstrate how a workflow engine with a TES backend can execute workflows on instances of different TES implementations.

We will use the [Snakemake](https://github.com/snakemake/snakemake) workflow engine.

### 3.1 Running Snakemake workflows

As for cwl-tes, we will reuse the "hash splitter" workflow, a simple
workflow with a scatter-gather step. The tasks of the workflow will be executed on individual TESK/Funnnel instances.
Existing files will be overwritten.

In [ ]:
export HOME=/tmp
for KEY in "${!TES_INSTANCES[@]}"; do
    TES="${TES_INSTANCES[$KEY]}"
    echo "Submitting task to $KEY ($TES)..."
    snakemake \
        --directory wf-hashsplitter \
        --snakefile wf-hashsplitter/Snakefile \
        --jobs 1 \
        --cores 1 \
        --tes "${TES%/}" \
        --forceall \
        --rerun-incomplete \
        --envvars HOME
    echo "================================================================================"
done
echo "DONE"

### 3.2 Task federation via Snakemake

Now let's point Snakemake to the proTES TES gateway:

In [ ]:
export HOME=/tmp
snakemake \
    --directory wf-federated \
    --snakefile wf-federated/Snakefile \
    --jobs 1 \
    --cores 1 \
    --tes "${TES_GATEWAY%/}"  \
    --forceall \
    --rerun-incomplete \
    --envvars HOME
echo "================================================================================"
echo "DONE"

### 3.2 Execute the federated workflow (S3 storage) using WESkit

First of all deploy WESkit using the following tutorial:
https://gitlab.com/one-touch-pipeline/weskit/documentation/-/blob/master/deployment/helm.md

Please make sure to activate the workflow_engine_parameters you indent to use. helm-deployment/configs/weskit/config.yaml.tpl
e.g. : "tes", "task-home", "data-aws-access-key-id", "data-aws-secret-access-key", "jobs", "cores"

After deployment of WESkit you need to configure your S3 storage and submit your request using cURL:


In [ ]:
Test deployment with: curl --ipv4 https://192.168.60.2:30132/ga4gh/wes/v1/service-info --cacert /home/$USER/weskit/helm-deployment/certs/weskit.crt

In [ ]:
run_id=$(curl -vvv\
 --ipv4 \
 --cacert /home/$USER/weskit/helm-deployment/certs/weskit.crt \
 -X POST  https://192.168.60.2:30132/ga4gh/wes/v1/runs \
 --header 'Content-Type: multipart/form-data' \
 --header 'Accept: application/json' \
 -F workflow_params='{}' \
 -F workflow_type="SMK" \
 -F workflow_type_version="7.30.2" \
 -F workflow_url="file:wf-federated/Snakefile" \
 -F workflow_engine_parameters='{ 
                "tes": "https://vm4816.kaj.pouta.csc.fi", 
                "task-home": "HOME",
                "data-aws-access-key-id": "$ACCESS_KEY_ID",
                "data-aws-secret-access-key": "$SECRET_ACCESS_KEY",
                "jobs": "1",
                "cores": "1"
            }'\
  | jq -r .run_id)

Feel free to adjust the "workflow_engine_parameters" to your needs.

In [ ]:
curl \
 --ipv4 \
 --cacert /home/$USER/weskit/helm-deployment/certs/weskit.crt \
 -X GET  https://192.168.60.2:30132/ga4gh/wes/v1/runs/$run_id \
 --header 'Content-Type: multipart/form-data' \
 --header 'Accept: application/json' 